# Jores et al 2021 Evaluation 
**Authorship:**
Adam Klie, *08/12/2022*
***
**Description:**
Notebook to perform a brief evaluation of trained models on the Jores et al (2021) dataset.
***

In [ ]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [112]:
# General imports
import os
import glob
import torch
import numpy as np
import xarray as xr

# EUGENe imports and settings
from eugene import models
from eugene import settings
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/jores21"
settings.output_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/output/fix_full/jores21"
settings.logging_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/logs/fix_full/jores21"
settings.config_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21"

# EUGENe packages
import seqdata as sd

# For illustrator editing
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Load in the `leaf`, `proto` and `combined` test `SeqData`s 

In [113]:
sdata_leaf = sd.open_zarr(os.path.join(settings.dataset_dir, "jores21_leaf_test.zarr"))
sdata_proto = sd.open_zarr(os.path.join(settings.dataset_dir, "jores21_proto_test.zarr"))
def concat_seqdatas(seqdatas, keys):
    for i, s in enumerate(seqdatas):
        s["batch"] = keys[i]
    return xr.concat(seqdatas, dim="_sequence")
sdata_combined = concat_seqdatas([sdata_leaf, sdata_proto], ["leaf", "proto"])

# Get test set predictions for each model

In [130]:
# Train 5 models with 5 different random initializations for each dataset and mopdel architecture
test_sets = {"proto": sdata_proto}#, "proto": sdata_proto, "combined": sdata_combined}
configs = ["cnn.yaml", "hybrid.yaml", "jores21_cnn.yaml", "deepstarr.yaml"]
trials = 3
for test_set in test_sets:
    sdata = test_sets[test_set]
    # Make an output directory for this dataset if it doesn't exist
    if not os.path.exists(os.path.join(settings.output_dir, test_set)):
        os.makedirs(os.path.join(settings.output_dir, test_set))
    for config in configs:
        model_name = config.split(".")[0]
        for trial in range(1, trials+1):
        
            # Print the model name
            print(f"{test_set} {model_name} trial {trial}")

            # Grab the best model from that training run
            model_file = glob.glob(os.path.join(settings.logging_dir, model_name, f"{test_set}_trial_{trial}", "checkpoints", "*"))[0]
            model = models.load_config(config_path=config)
            best_model = models.SequenceModule.load_from_checkpoint(model_file, arch=model.arch)

            # Grab the predictions on the test set
            ohe_seqs = sdata["ohe_seq"].to_numpy().transpose(0, 2, 1)
            preds = best_model.predict(ohe_seqs, batch_size=128).detach().numpy().squeeze()
            sdata[f"{model_name}_{test_set}_trial_{trial}_preds"] = xr.DataArray(preds, dims=["_sequence"])

    pred_keys = [k for k in sdata.data_vars.keys() if "preds" in k]
    target_keys = ["enrichment"]
    sdata[["id", *target_keys, *pred_keys]].to_dataframe().to_csv(os.path.join(settings.output_dir, test_set, f"jores21_{test_set}_test_predictions.tsv"), sep="\t", index=False)
    sdata.to_zarr(os.path.join(settings.output_dir, test_set, f"jores21_{test_set}_test_predictions.zarr"), mode="w")

proto cnn trial 1


[rank: 0] Global seed set to 1


Predicting on batches:   0%|          | 0/59 [00:00<?, ?it/s]

proto cnn trial 2


[rank: 0] Global seed set to 2


...

[rank: 0] Global seed set to 2


Predicting on batches:   0%|          | 0/59 [00:00<?, ?it/s]

proto deepstarr trial 3


[rank: 0] Global seed set to 3


Predicting on batches:   0%|          | 0/59 [00:00<?, ?it/s]

In [46]:
sdata.chunk()

<xarray.Dataset>
Dimensions:                         (_sequence: 14749, length: 170, _ohe: 4)
Dimensions without coordinates: _sequence, length, _ohe
Data variables: (12/28)
    batch                           (_sequence) <U5 dask.array<chunksize=(14749,), meta=np.ndarray>
    enrichment                      (_sequence) float64 dask.array<chunksize=(7154,), meta=np.ndarray>
    gene                            (_sequence) <U1 dask.array<chunksize=(7154,), meta=np.ndarray>
    id                              (_sequence) <U8 dask.array<chunksize=(7154,), meta=np.ndarray>
    ohe_seq                         (_sequence, length, _ohe) uint8 dask.array<chunksize=(7154, 170, 4), meta=np.ndarray>
    seq                             (_sequence) <U1 dask.array<chunksize=(7154,), meta=np.ndarray>
    ...                              ...
    jores21_cnn_leaf_trial_5_preds  (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
    deepstarr_leaf_trial_1_preds    (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
    deepstarr_leaf_trial_2_preds    (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
    deepstarr_leaf_trial_3_preds    (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
    deepstarr_leaf_trial_4_preds    (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
    deepstarr_leaf_trial_5_preds    (_sequence) float32 dask.array<chunksize=(14749,), meta=np.ndarray>
Attributes:
    max_jitter:  0

---